# Análise de Dados e Gráficos com *R*

O objetivo deste exercício é demonstrar algumas funcionalidades da linguagem *R* para análise estatística e geração de gráficos.

As dependências de software já estão instaladas na imagem *Docker* disponibilizada no repositório do minicurso, que também determina as versões dos pacotes que vamos usar. Rode a célula abaixo para carregar os pacotes que vamos usar, e listar o ambiente disponível.

Apesar de não garantir a reprodutibilidade, poder controlar as versões do sistema operacional, da linguagem *R*, e de cada pacote que usamos ajuda a compartilhar código, facilita o trabalho colaborativo, e ajuda a resolver problemas no futuro, quando as versões podem mudar.

In [ ]:
# Pacotes usados
library(dplyr) # Manipulação de dados
library(tidyr) # Manipulação de dados
library(ggplot2) # Geração de gráficos
library(datasauRus) # Dados utilizados

sessionInfo() # Lista ambiente desta seção

## O Quarteto de Anscombe e o Datasaurus

Agora já devemos estar com o conjunto de dados carregado. Esse conjunto de dados foi elaborado para demonstrar que sempre devemos desconfiar de sínteses feitas a partir de um conjunto de dados, mesmo quando os desvios padrão e intervalos de confiança são incluídos. O conjunto de dados que vamos usar é uma extensão dos conceitos discutidos no [Quarteto de Anscombe](https://pt.wikipedia.org/wiki/Quarteto_de_Anscombe), representados pela tabela retirada da Wikipédia e reproduzida abaixo:

![Anscombe](img/anscombe.png)

Veja que todas as tentativas de sintetizar as diferenças entre esse conjunto de dados falham em demonstrar a diferença que podemos observar com os gráficos.

Neste exercício, nós vamos fazer uma análise estatística "do avesso". Vamos começar com medidas de sumário e depois verificar o comportamento dos dados com gráficos. Isso é exatamente o inverso do que devemos fazer num cenário real.

### Conjuntos de dados no *R*

Antes de começar, vamos observar alguns conjuntos de dados bastante simples. As três funções abaixo geram dados *y* normalmente distribuídos, positivamente correlacionados, e negativamente correlacionados com um vetor de entrada *x*.

In [ ]:
random_y <- function(x) {
    rnorm(length(x))
}

poscor_y <- function(x) {
    x + rnorm(length(x), mean = 0, sd = 0.05)
}

negcor_y <- function(x) {
    -x + rnorm(length(x), mean = 0, sd = 0.05)
}

samples = 30
xs = seq(0, 1, length.out = samples)

first_df = data.frame(x = rep(xs, 3)) %>%
    mutate(y = c(random_y(xs), poscor_y(xs), negcor_y(xs)),
           dataset = c(rep("A", samples),                                
                       rep("B", samples),
                        rep("C", samples)))

Fazendo gráficos de dispersão dos *x* e *y* gerados, vemos:

In [ ]:
options(repr.plot.width = 22, repr.plot.height = 8)

ggplot() +
    geom_point(data = first_df,
              aes(x, y),
              size = 3) +
    facet_wrap(dataset ~ .) +
    theme_bw(base_size = 35)

Calculando algumas sínteses estatísticas comuns, temos a tabela:

In [ ]:
summary_first_df = first_df %>%
    group_by(dataset) %>%
    summarize(mean_x = mean(x),
              mean_y = mean(y),
              ci_95_x = (1.96 * sd(x)) / sqrt(n()),
              ci_95_y = (1.96 * sd(y)) / sqrt(n()),
              cor_pearson = cor(x, y)) %>%
    ungroup()

summary_first_df

### O Datasaurus

Agora, podemos começar a análise "do avesso", usando o Datasaurus. Primeiro, vamos usar a função `str` para começar a entender o conjunto de dados que temos à mão:

In [ ]:
str(data.frame(datasaurus_dozen))

Para facilitar a leitura da saída, transformamos a variável `datasaurus_dozen`, definida pelo pacote `datasauRus`, num `data.frame`. Experimente a saída sem usar o DataFrame. Qual o tipo dos dados? De qual pacote esse tipo vem?

Vemos que temos 1846 observações, correspondentes a cada linha, e três variáveis, correspondentes às colunas. Um DataFrame é basicamente uma tabela, e é padrão colocar as observações, ou experimentos, nas linhas, e as variáveis nas colunas.

Se quisermos saber quais são os valores da coluna `dataset`, podemos fazer:

In [ ]:
unique(datasaurus_dozen$dataset)

Podemos também observar as primeiras linhas desse DataFrame com a função `head`:

In [ ]:
datasaurus_dozen %>% head(n = 10)

Note que estamos usando o operador `%>%`, ou *pipe*, para passar a variável `datasaurus_dozen` como primeiro argumento da função `head`. Podemos fazer o mesmo com a função `sample_n`, para observar uma amostra aleatória do DataFrame:

In [ ]:
datasaurus_dozen %>% sample_n(size = 10)

Agora, vamos criar um novo DataFrame chamado `df`, contento as médias das colunas `x` e `y`, e outras sínteses:

In [ ]:
df = datasaurus_dozen %>%
    group_by(dataset) %>%
    summarize(mean_x = mean(x),
              mean_y = mean(y),
              ci_95_x = (1.96 * sd(x)) / sqrt(n()),
              ci_95_y = (1.96 * sd(y)) / sqrt(n()),
              cor_pearson = cor(x, y)) %>%
    ungroup()

df

Estamos usando a função `group_by`, do pacote `dplyr`, para agrupar as linhas do DataFrame e calcular as estatísticas separadamente. O que é o intevalo de confiança a 95%? E o índice de correlação de Pearson?

Relembrando, podemos estimar o índice de correlação de Pearson, escrito

$$
\rho = \dfrac{\text{cov}(X,Y)}{\sigma_x \sigma_y}\text{,}
$$

usando a fórmula

$$
r = \dfrac{{}\sum_{i=1}^{n} (x_i - \overline{x})(y_i - \overline{y})}
{\sqrt{\sum_{i=1}^{n} (x_i - \overline{x})^2(y_i - \overline{y})^2}}\text{,}
$$

e podemos estimar o desvio padrão fazendo

$$
\sigma = \sqrt{\dfrac{1}{N} \sum_{i=1}^N (x_i - \mu)^2}\text{.}
$$

### Questão

Qual é a sua interpretação da tabela com sínteses estatísticas? Baseando-se nela, você diria que há diferenças entre os `datasets`? Quais?
Nesse ponto da análise, o que você faria para explorar melhor os dados?

### Quais gráficos podem ajudar?

Vamos colocar os dados em gráficos. Quais gráficos devemos fazer? Se apenas representarmos os mesmos dados da tabela, não vamos aprender nada novo. Os dois gráficos abaixo apenas reforçam a impressão de que os conjuntos de dados são idênticos.

In [ ]:
options(repr.plot.width = 20, repr.plot.height = 10)

ggplot(df) +
    geom_point(aes(x = dataset, y = mean_x)) +
    geom_errorbar(aes(ymin = mean_x - ci_95_x,
                     ymax = mean_x + ci_95_x,
                     x = dataset)) +
    ylim(0, NA) +
    ylab("Média dos x + CI") +
    theme_bw(base_size = 35) +
    theme(legend.position = "none",
        axis.text.x = element_text(angle = 30, hjust = 1))

In [ ]:
options(repr.plot.width = 20, repr.plot.height = 10)

ggplot(df) +
    geom_point(aes(x = dataset, y = mean_y)) +
    geom_errorbar(aes(ymin = mean_y - ci_95_y,
                     ymax = mean_y + ci_95_y,
                     x = dataset)) +
    ylim(0, NA) +
    ylab("Média dos y + CI") +
    theme_bw(base_size = 35) +
    theme(legend.position = "none",
        axis.text.x = element_text(angle = 30, hjust = 1))

## Desvendando o Datasaurus

O [Datasaurus Dozen](http://www.thefunctionalart.com/2016/08/download-datasaurus-never-trust-summary.html) é uma extensão do conceito apresentado no [Quarteto de Anscombe](https://pt.wikipedia.org/wiki/Quarteto_de_Anscombe). Se fizermos um gráfico de dispersão com as variáveis $x$ e $y$, veremos que cada um dos 13 conjuntos de dados (12 + o Datasaurus) é completamente diferente!

In [ ]:
options(repr.plot.width = 20, repr.plot.height = 18)

ggplot() +
    geom_hline(data = df,
               size = 1.4,
               linetype = 2,
               alpha = 1,
               color = "gray80",
               aes(yintercept = mean_y)) +
    geom_vline(data = df,
               size = 1.4,
               linetype = 2,
               alpha = 1,
               color = "gray80",
               aes(xintercept = mean_x)) +
    geom_point(data = datasaurus_dozen,
               size = 2,
               aes(x = x, y = y, color = dataset)) +
    theme_bw(base_size = 30) +
    theme(legend.position = "none",
         panel.grid = element_blank(),
         strip.background = element_rect(fill = "transparent")) +
    facet_wrap(~ dataset, ncol = 4)

Os painéis da figura acima mostram cada um dos conjuntos de dados, com linhas tracejadas marcando as médias dos dados em cada eixo.

Esses conjuntos foram criados a partir do Datasaurus (o conjunto "Dino"), através da perturbação progressiva dos pontos, mas mantendo um conjunto de métricas estatísticas constante. O processo é descrito em mais detalhes [neste post](https://www.autodesk.com/research/publications/same-stats-different-graphs), e o objetivo é mostrar que toda análise estatística deve começar por gráficos bem escolhidos.

## Exercício: Análise Estatística

Agora que já compreendemos que existem diferenças, procure encontrar uma ou mais formas de distinguir os conjuntos de dados no Datasaurus Dozen usando análises estatísticas.

Você pode copiar e modificar os trechos de código acima, pode procurar na documentação dos pacotes /R/ que usamos (ou outros), e também pode conversar com outros participantes e com o instrutor do curso.

Veja também as duas sugestões abaixo.

### Modelos Lineares

Aqui, estamos usando o `geom_smooth` da bilioteca `ggplot2`. No entanto, parece que todas as regressões são iguais! Como prosseguir?

In [ ]:
options(repr.plot.width = 20, repr.plot.height = 18)

ggplot() +
    geom_point(data = datasaurus_dozen,
               size = 2,
               aes(x = x, y = y, color = dataset)) +
    geom_smooth(data = datasaurus_dozen,
                method = "lm",
                formula = y ~ x,
                aes(x = x, y = y, color = dataset)) +
    theme_bw(base_size = 30) +
    theme(legend.position = "none",
         panel.grid = element_blank(),
         strip.background = element_rect(fill = "transparent")) +
    facet_wrap(~ dataset, ncol = 4)

### Histogramas

Modifique os exemplos anteriores e use o `geom_histogram` do pacote `ggplot2`. Elabore um argumento sobre o que você interpreta a partir das distribuições produzidas para cada conjunto de dados.

In [ ]:
options(repr.plot.width = 20, repr.plot.height = 18)

ggplot() +
    geom_histogram(data = datasaurus_dozen,               
                   aes(x = x, fill = dataset),                              
                   bins = 35) +
    theme_bw(base_size = 30) +
    theme(legend.position = "none",
         panel.grid = element_blank(),
         strip.background = element_rect(fill = "transparent")) +
    facet_wrap(~ dataset, ncol = 4)